# OSN postprocessing

Spracovanie výstupu algoritmu do formy využiteľnej pre ďalšie analýzy

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
# Aby fungovalo nacitanie spolocnych funkcii, je nutne ich nainstalovat cez `pip install .` na urovni, kde sa nachadza pyproject.toml

from OSN_common.nacitanie_dat import nacitaj_vsetku_starostlivost, nacitaj_vystup_algoritmu, nacitaj_vsetku_starostlivost_s_ms, nacitaj_zoznam_ms, nacitaj_programovy_profil, nacitaj_zoznam_nemocnic, nacitaj_siet_nemocnic
from OSN_common.pomocne_funkcie import prirad_vekovu_kategoriu, zisti_uroven_ms


In [ ]:
_rok = 2023
_verzia = 'v2024.2'
_prepinace_algoritmu = '-vnd'

# Pripojenie výstupu algoritmu ku všetkej starostlivosti

In [ ]:
vystup = nacitaj_vystup_algoritmu(_rok, _verzia, _prepinace_algoritmu)
starostlivost = nacitaj_vsetku_starostlivost(_rok)

In [ ]:
final = starostlivost.merge(vystup, on='id_hp', how='left')

## Pripojenie úrovní medicínskych služieb

In [ ]:
final['vekova_kategoria'] = final['vek_roky'].apply(prirad_vekovu_kategoriu)
final['ms_list'] = final['ms'].apply(lambda s: s.split('~') if pd.notna(s) else s)


In [ ]:
zoznam_ms = nacitaj_zoznam_ms(_verzia)
zoznam_ms = zoznam_ms[~zoznam_ms['zdielana_ms']]

In [ ]:
final_exploded = final[['id_hp', 'pzs_8', 'vekova_kategoria', 'ms_list']].explode('ms_list')
final_exploded = final_exploded.merge(zoznam_ms.filter(regex='kod_ms|uroven_'), left_on='ms_list', right_on='kod_ms', how='left')


In [ ]:
final_exploded['uroven_ms'] = final_exploded.progress_apply(zisti_uroven_ms, axis=1).astype('Int16')

In [ ]:
urovne_ms = final_exploded.groupby('id_hp')['uroven_ms'].progress_apply(lambda z: '~'.join([str(x) for x in z]))

In [ ]:
final = final.merge(urovne_ms, on='id_hp')
final['hlavna_ms'] = final['ms_list'].str[0]

## MS na úrovne programu 

In [ ]:
programovy_profil = nacitaj_programovy_profil(2024, _verzia)

In [ ]:
final['cislo_programu'] = final['hlavna_ms'].str.split('-').str[0].str[1:].astype(int)
final['uroven_hlavnej_ms'] = final['uroven_ms'].str.split('~').str[0]

In [ ]:
zoznam_nemocnic = nacitaj_zoznam_nemocnic()

In [ ]:
kraje = zoznam_nemocnic['kraj'].unique()

In [ ]:
final = final.reset_index().merge(nacitaj_siet_nemocnic(_rok).index.to_frame().set_index('pzs_6')[['uroven_nemocnice']], how='left', left_on='pzs_6', right_index=True)
final = final.merge(zoznam_nemocnic[['pzs_6', 'kraj']], how='left', on='pzs_6')
final = final.merge(programovy_profil[['cislo_programu', 'nazov_programu']].drop_duplicates(), how='left', on='cislo_programu')

In [ ]:
pivot = pd.pivot_table(final, values='id_hp', index=['cislo_programu', 'nazov_programu'], columns=['uroven_nemocnice', 'uroven_hlavnej_ms'], aggfunc='count').fillna(0).astype(int)

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

template_name = f'TEMPLATE_pocet_MS_na_uroven_programu_iba_hlavne_{_rok}_{_verzia}.xlsx'
output_name = f'Vystupy/pocet_MS_na_uroven_programu_iba_hlavne_{_rok}_{_verzia}_podla_krajov.xlsx'

wb = load_workbook(template_name)

for kraj in kraje:
    ws = wb.create_sheet(kraj)
    
    pivot = pd.pivot_table(final[final['kraj'] == kraj], values='id_hp', index=['cislo_programu', 'nazov_programu'], columns=['kraj', 'uroven_nemocnice', 'uroven_hlavnej_ms'], aggfunc='count', fill_value=0).fillna(0).astype(int)
    
    for r in dataframe_to_rows(pivot):
        ws.append(r)
        
    wb.save(output_name)



pzs_6 = 'P43059'

ws = wb.create_sheet(pzs_6)

pivot = pd.pivot_table(final[final['pzs_6'] == pzs_6], values='id_hp', index=['cislo_programu', 'nazov_programu'], columns=['pzs_6', 'uroven_nemocnice', 'uroven_hlavnej_ms'], aggfunc='count').fillna(0).astype(int)

for r in dataframe_to_rows(pivot):
    ws.append(r)
    
wb.save(output_name)

# Export

In [ ]:
vystupne_stlpce = ['id_hp', 'vek_roky', 'vek_dni', 'hmotnost', 'upv', 'diagnozy', 'vykony', 'drg', 'erv', 'typ_starostlivosti', 'typ_hospitalizacie', 'druh_prijatia', 'dovod_prijatia', 'dovod_prepustenia', 'posledna_dgn_prijem', 'posledna_dgn_prepustenie', 'posledny_pohyb_poistenca', 'diagnozy_z_01', 'vykony_z_01', 'novorodenec', 'obdobie_od', 'obdobie_do', 'datum_od', 'datum_do', 'osetrovacia_doba', 'id_poistenca', 'datum_narodenia', 'pohlavie', 'kod_pobytu', 'kod_zp', 'pzs_12', 'IDENTIFZAR', 'pzs_8', 'pzs_6', 'pzs_ico', 'id_hp_pzs', 'ms', 'uroven_ms']

final.filter(vystupne_stlpce).to_csv(f'Vystupy/osn_vsetka_starostlivost_{_rok}_ms_{_verzia}.csv', sep=';', index=False)


In [ ]:
final.groupby(['hlavna_ms', 'pzs_8'])['id_hp'].count().rename('pocet_hospitalizacnych_pripadov').to_csv(f'Vystupy/pocet_MS_na_nemocnicu_iba_hlavne_{_rok}_{_verzia}.csv')
final_exploded.rename({'ms_list': 'ms'}, axis=1).groupby(['ms', 'pzs_8'])['id_hp'].count().rename('pocet_hospitalizacnych_pripadov').to_csv(f'Vystupy/pocet_MS_na_nemocnicu_{_rok}_{_verzia}.csv')


In [ ]:
final.groupby(['hlavna_ms', 'pzs_ico', 'typ_hospitalizacie'])['id_hp'].count().rename('pocet_hospitalizacnych_pripadov').to_csv(f'Vystupy/pocet_MS_na_ico_iba_hlavne_{_rok}_{_verzia}.csv')

## Kontrola s pôvodným výstupom

In [ ]:
final_old = nacitaj_vsetku_starostlivost_s_ms(_rok, _verzia)

merged = final.merge(final_old[['id_hp', 'ms']], on='id_hp', how='outer')

In [ ]:
len(merged[merged['ms_x'] != merged['ms_y']])

In [ ]:
a=merged[merged['ms_x'] != merged['ms_y']]